# Vehicle Routing Problem with Time Windows (VRPTW)
Solving CRVPPTW with OR-Tools

**Aim: Efficient route optimisation for multi-stop journals to minimise travel time, and considering road restrictions, time contraints, etc.**

# Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import folium.plugins as plugins
from folium.features import DivIcon

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import random
import re
import json
import time
from datetime import datetime
import itertools
import functools
import operator

import openrouteservice
import WazeRouteCalculator as wrc
import geocoder

import json
import requests
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, Point, MultiPoint

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display
import sys
import os

os.getcwd()


In [2]:
# Model parameters

# set individual maximum distance travel of any truck (in km)
max_travel_distance = 20000  

# penalty value for each unmatched point
penalty_value = 1000000  

# span coefficient for optimizing variance of distances per truck
globalSpanCostCoefficient = 1000  

# impossible value for trucks to avoid points that has missing routes
impossible_value = 1000 
impossible = impossible_value

# notes:
# max travel distance could be any big number
# impossible value >= max travel distance  | "impossible"
# penalty value must not be too low or else routes will start to have 0 travel

# Routing using CVRPTW - Capacitated Vehicle Routing Problem with Time Windows

## One customer

### Inputs

Inputs (Customer Side)
- Preferred date range
    - Delivery by date
- Vehicles for Job Order
    - Per Vehicle (If you just want 1 vehicle, amenable to many vehicles)
        - Type (e.g. Big Truck, Small truck, Motorbike, Sedan, Van)
        - Size (capacity)
        - 
- Trip
    - Starting Location of Trip (LongLat)*
        - Location (LongLat)
        - Delivery weight (kg)
        - Arrival time (e.g. 6:00am)
        - Leave-by time (e.g. 7:00am)
    - Per Stop (define stop i_n with n is nth stop)
        - Delivery type (Pick-up, Drop-off)
        - Location (LongLat)
        - Delivery weight (kg)
        - If Drop-off, drop-off location
        - Time-window time open (e.g. 8:00am)
        - Time-window time close (e.g. 5:00pm)

In [4]:
# vehicle dictionary
open_var = 1000

fleet = {
    "sedan": {"max_weight": 200, "max_l": 3.5, "max_w": 2, "max_h": 2.5},
    "l300": {
        "max_weight": 1000,
        "max_l": 10,
        "max_w": 4.5,
        "max_h": 2.5,
    },
    "pickup": {
        "max_weight": 1000,
        "max_l": 5,
        "max_w": 5,
        "max_h": open_var,
    },
    "fwd_truck": {
        "max_weight": 7000,
        "max_l": 18,
        "max_w": 6,
        "max_h": 7,
    },
}

In [5]:
# sample dictionary
input_dict = {
    "starting_location": {
        # 'lat': 12,
        # 'lon': 121,
        "start_address": "69 J.Lianes Escoda, Manila, Metro Manila, Philippines",
        # 'total_weight': 1000,
        "vehicle": "pickup",
        "pickup_time": pd.to_datetime("28 apr 2023 6:00am", format="%d %b %Y %I:%M%p"),
    },
    "delivery_details": {
        "a": {
            # 'lat': 13,
            # 'lon': 122,
            "location": "Pasig, Metro Manila, Philippines",
            "weight": 400,
            "buffer": 20,
            "dev_pickup": "delivery",
            "time_window_order": [
                pd.to_datetime("28 apr 2023 11:00am", format="%d %b %Y %I:%M%p"),
                pd.to_datetime("29 apr 2023 6:00pm", format="%d %b %Y %I:%M%p"),
            ],
        },
        "b": {
            # 'lat': 13,
            # 'lon': 122,
            "location": "5 Mercury Ave, Bagumbayan, Quezon City, 1100 Metro Manila, Philippines, Metro Manila",
            "weight": 100,
            "buffer": 30,
            "dev_pickup": "delivery",
            "time_window_order": [
                pd.to_datetime("28 apr 2023 10:00am", format="%d %b %Y %I:%M%p"),
                pd.to_datetime("28 apr 2023 6:00pm", format="%d %b %Y %I:%M%p"),
            ],
        },
        "c": {
            # 'lat': 13,
            # 'lon': 122,
            "location": "San Rafael Village, Navotas, Metro Manila, Philippines",
            "weight": 250,
            "buffer": 30,
            "dev_pickup": "delivery",
            "time_window_order": [
                pd.to_datetime("28 apr 2023 9:00am", format="%d %b %Y %I:%M%p"),
                pd.to_datetime("28 apr 2023 4:00pm", format="%d %b %Y %I:%M%p"),
            ],
        },
        "d": {
            # 'lat': 13,
            # 'lon': 122,
            "location": "2821 F. Manalo St., 898 Z Punta 100 Sta Ana, Manila, 1009 Metro Manila, Philippines, Metro Manila",
            "weight": 50,
            "buffer": 15,
            "dev_pickup": "delivery",
            "time_window_order": [
                pd.to_datetime("28 apr 2023 6:30am", format="%d %b %Y %I:%M%p"),
                pd.to_datetime("28 apr 2023 6:00pm", format="%d %b %Y %I:%M%p"),
            ],
        },
        "e": {},
    },
}

In [6]:
# compile delivery inputs
del_dev = input_dict["delivery_details"]

# Check how many actual orders are there
orders = [k for k, v in del_dev.items() if v]

# Compute variables
n_trucks, n_depots = 1, 1
n_orders = len(orders)
n_totalTrucks = n_trucks * n_depots
truck_starts, truck_ends = [0], [0]

### Run routing process 

In [7]:
# Define ORS API
ors_key = "212415"

In [8]:
def geocode(address):
    """
    Uses Waze to convert an address string into a set of coordinates.
    Parameters:
    address (str): The address to convert to coordinates.
    Returns:
    List[float]: A list of two float values representing the longitude and latitude of the address.
    """
    route = wrc.WazeRouteCalculator(
        address, "Manila, Philippines", region="AU", vehicle_type="TAXI"
    )
    coords = route.address_to_coords(address)
    co = [coords["lon"], coords["lat"]]

    return co

def already_coords(address):
    """Check if input is a coordinate list.
    Parameters:
    address (str): the address for checking
    Returns:
    True if the address is in coordinate, else returns False
    """
    COORD_MATCH = re.compile(
        r"^([-+]?)([\d]{1,2})(((\.)(\d+)(,)))(\s*)(([-+]?)([\d]{1,3})((\.)(\d+))?)$"
    )

    try:
        m = re.search(COORD_MATCH, address)
        return m is not None
    except TypeError:
        return True

def geocode_list(locations):
    """Given a list of locations, return a list of coordinates"""
    return [
        geocode(address) if not already_coords(address) else address
        for address in locations
    ]

def distance_matrix(arr_depots, arr_orders):
    """Given a list of depots and orders, create a distance matrix."""

    # Order points to depot(s) first and then order(s)
    arr_points = arr_depots + arr_orders

    ors_params = {
        "locations": arr_points,
        "metrics": ["distance", "duration"],
        "units": "km",
        "profile": "driving-hgv",
        "optimized": True,
    }
    ors = openrouteservice.Client(key=ors_key)

    try:
        ors_matrix = openrouteservice.distance_matrix.distance_matrix(ors, **ors_params)
    except openrouteservice.exceptions.ApiError as e:
        raise ValueError(f"Error calling ORS API: {e}") from None

    # Convert ORS matrix results to pandas dataframes
    df_matrix_distance = pd.DataFrame(ors_matrix["distances"])
    df_matrix_duration = pd.DataFrame(ors_matrix["durations"])

    # Replace zeros with impossible value
    matrix_distance = np.where(
        df_matrix_distance.values == 0, impossible_value, df_matrix_distance.values
    )
    matrix_duration = (
        np.where(
            df_matrix_duration.values == 0, impossible_value, df_matrix_duration.values
        )
        / 60
    )

    # Set diagonal to zero
    np.fill_diagonal(matrix_distance, 0)
    np.fill_diagonal(matrix_duration, 0)

    matrix_distance = matrix_distance.tolist()
    matrix_duration = matrix_duration.tolist()

    return arr_points, matrix_distance, matrix_duration

def add_zero_dist_point(list_matrix):
    """Returns an additional column and row, where the values are zero, except for the diagonal value.
    This is useful where the distance from a point back to itself is always zero.
    """
    return np.pad(list_matrix, ((1, 0), (1, 0)), "constant")


def data_model(
    matrix_distance,
    matrix_duration,
    truck_starts,
    truck_ends,
    weight_vehicle,
    time_window_orders,
    time_window_trucks,
    weights,
    buffer=None,
    depot=False,
):
    """Stores data for VRP."""
    n = len(matrix_distance)
    if not buffer:
        buffer = [0] * (n - 1)
    waiting_time = [0] + buffer
    waiting_matrix = np.full((n, n), np.add.outer(waiting_time, waiting_time))
    np.fill_diagonal(waiting_matrix, 0)
    waiting_durations = np.add(matrix_duration, waiting_matrix).tolist()

    data = {
        "buffer": buffer,
        "distance_matrix": add_zero_dist_point(matrix_distance),
        "duration_matrix": add_zero_dist_point(waiting_durations),
        "num_vehicles": len(truck_starts),
        "starts": [1] if depot else truck_starts,
        "ends": [0] if depot else truck_ends,
        "vehicle_capacities": [weight_vehicle]
        if not isinstance(weight_vehicle, list)
        else weight_vehicle,
        "demands": [0] * (n_depots + 1) + weights,
        "time_windows_orders": time_window_orders,
        "time_windows_trucks": time_window_trucks,
    }

    return data

In [9]:
def time_orders(del_dev, time_min):
    """
    Returns a list that converts Timestamps into base minutes, with starting time synced to the start time from the starting location.

    Parameters
    ----------
    del_dev: sub-dictionary of delivery details
    """
    if del_dev is None:
        return [], [time_min]

    time_orders = [
        [int((i - time_min).total_seconds() // 60) for i in value["time_window_order"]]
        for value in del_dev.values()
        if value
    ]
    time_list = [
        [time_min] + value["time_window_order"] for value in del_dev.values() if value
    ]

    out = [item for sublist in time_list for item in sublist]
    return time_orders, out


orders_time, time_list = time_orders(
    del_dev, input_dict["starting_location"]["pickup_time"]
)


In [10]:
def time_trucks(time_list):
    """
    Returns a list for the trucks' time windows. Assumes truck availability spans the availability of all orders.

    Parameters
    ----------
    time_list: list of Timestamps
    """
    time_min = min(time_list)
    time_range = (max(time_list) - min(time_list)).total_seconds() / 60
    time_trucks = [[0, int(time_range)] for _ in range(n_totalTrucks)]

    return time_trucks, time_min


trucks_time, min_time = time_trucks(time_list)

In [11]:
vehicle = input_dict["starting_location"]["vehicle"]
weight_vehicle = fleet[vehicle]["max_weight"]
vehicle, weight_vehicle

starting = [input_dict["starting_location"]["start_address"]]
arr_starting = geocode_list(starting)

# Get location from locations, buffer
locations = []
buffers = []
dp = []
weights = []
for i in del_dev:
    if not del_dev[i]:
        continue
    locations.append(del_dev[i]["location"])
    buffers.append(del_dev[i]["buffer"])
    dp.append(del_dev[i]["dev_pickup"])
    weights.append(del_dev[i]["weight"])

arr_locations = geocode_list(locations)

# Distance matrix
arr_points, distance, duration = distance_matrix(arr_starting, arr_locations)

starting, arr_starting, locations, arr_locations, buffers, dp, weights, distance, duration

(['69 J.Lianes Escoda, Manila, Metro Manila, Philippines'],
 [[120.96398162841797, 14.634960174560547]],
 ['Pasig, Metro Manila, Philippines',
  '5 Mercury Ave, Bagumbayan, Quezon City, 1100 Metro Manila, Philippines, Metro Manila',
  'San Rafael Village, Navotas, Metro Manila, Philippines',
  '2821 F. Manalo St., 898 Z Punta 100 Sta Ana, Manila, 1009 Metro Manila, Philippines, Metro Manila'],
 [[121.09098815917969, 14.576546669006348],
  [121.07904052734375, 14.612759590148926],
  [120.94569396972656, 14.657938957214355],
  [121.01588439941406, 14.583112716674805]],
 [20, 30, 30, 15],
 ['delivery', 'delivery', 'delivery', 'delivery'],
 [400, 100, 250, 50],
 [[0.0, 22.54, 19.75, 5.08, 14.46],
  [22.13, 0.0, 5.82, 24.97, 11.85],
  [18.99, 6.88, 0.0, 21.84, 9.5],
  [3.91, 24.25, 21.47, 0.0, 15.85],
  [14.24, 11.61, 9.75, 16.85, 0.0]],
 [[0.0, 37.132666666666665, 31.6355, 8.840333333333332, 24.402333333333335],
  [37.20366666666666, 0.0, 10.602666666666666, 41.749, 21.805833333333332],
  

In [12]:
# Visualization -sanity check
# Turn to Shapely objects
gdf_depots = gpd.GeoSeries([Point(x, y) for x, y in arr_starting])
gdf_orders = gpd.GeoSeries([Point(x, y) for x, y in arr_locations])

centroid = (gdf_depots.append(gdf_orders)).unary_union.convex_hull.centroid
map = folium.Map(location=[centroid.y, centroid.x])

# Markers for depot (starting location)
[
    folium.Marker(
        location=[point.y, point.x],
        icon=folium.Icon(icon="fa-truck", prefix="fa", color="orange"),
    ).add_to(map)
    for point in gdf_depots
]

# Markers for locations
[folium.Marker(location=[point.y, point.x]).add_to(map) for point in gdf_orders]

map

In [13]:
data = data_model(
    distance,
    duration,
    truck_starts,
    truck_ends,
    weight_vehicle,
    orders_time,
    trucks_time,
    weights,
    buffer=buffers,
    depot=True,
)
data

{'buffer': [20, 30, 30, 15],
 'distance_matrix': array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  , 22.54, 19.75,  5.08, 14.46],
        [ 0.  , 22.13,  0.  ,  5.82, 24.97, 11.85],
        [ 0.  , 18.99,  6.88,  0.  , 21.84,  9.5 ],
        [ 0.  ,  3.91, 24.25, 21.47,  0.  , 15.85],
        [ 0.  , 14.24, 11.61,  9.75, 16.85,  0.  ]]),
 'duration_matrix': array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        , 57.13266667, 61.6355    , 38.84033333,
         39.40233333],
        [ 0.        , 57.20366667,  0.        , 60.60266667, 91.749     ,
         56.80583333],
        [ 0.        , 60.9485    , 62.411     ,  0.        , 95.49383333,
         62.792     ],
        [ 0.        , 36.64583333, 89.534     , 94.03683333,  0.        ,
         71.19133333],
        [ 0.        , 39.2315    , 56.8755    , 63.45683333, 73.46766667,
          0.        ]]),
 'num_vehicles': 1,
 'starts': [1],

In [14]:
def print_solution(data, manager, routing, solution):
    """
    Prints solution on console.

    Parameters
    ----------
    data: __
    manager: __
    routing: __
    solution: __
    """
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_distance = 0
    total_load = 0
    total_time = 0

    for vehicle_id in range(data["num_vehicles"]):
        # loop over number of vehicles
        index = routing.Start(vehicle_id)
        plan_output = "Route for vehicle {}:\n".format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)

            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += " {0} Load({1}) Time({2}, {3}) -> ".format(
                node_index, route_load, solution.Min(time_var), solution.Max(time_var)
            )
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )

        time_var = time_dimension.CumulVar(index)
        plan_output += " {0} Load({1}), Time({2}, {3})\n".format(
            manager.IndexToNode(index),
            route_load,
            solution.Min(time_var),
            solution.Max(time_var),
        )
        plan_output += "Distance of the route: {}km\n".format(route_distance)
        plan_output += "Load of the route: {}\n".format(route_load)
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        total_time += solution.Min(time_var)
    print("Total distance of all routes: {}km".format(total_distance))
    print("Total load of all routes: {}".format(total_load))
    print(f"Total time of all routes: {total_time}min")


def get_vehicle_data(data, manager, routing, solution):
    """
    Returns a dictionary of routes, cumulative capacities, cumulative distances, and cumulative durations per vehicle in the solution.

    Parameters
    ----------
    data: __
    manager: __
    routing: __
    solution: __
    """

    res = {}
    res["objective"] = solution.ObjectiveValue()
    res["vehicle_data"] = {}
    time_dimension = routing.GetDimensionOrDie("Time")

    for vehicle_id in range(data["num_vehicles"]):
        # loop over number of vehicles
        vehicle_route = []
        vehicle_cum_capacity = []
        vehicle_cum_distance = []
        vehicle_cum_time = []

        index = routing.Start(vehicle_id)

        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            vehicle_cum_time.append(solution.Min(time_var))
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]

            vehicle_route.append(node_index)
            vehicle_cum_capacity.append(route_load)

            previous_index = index
            index = solution.Value(routing.NextVar(index))

            vehicle_cum_distance.append(route_distance)

            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )

        vehicle_route.append(manager.IndexToNode(index))
        vehicle_cum_capacity.append(route_load)
        vehicle_cum_distance.append(route_distance)
        vehicle_cum_time.append(solution.Min(time_var))

        res["vehicle_data"][vehicle_id] = {
            "route": vehicle_route,
            "cum_capacity": vehicle_cum_capacity,
            "cum_distance": vehicle_cum_distance,
            "cum_time": vehicle_cum_time,
        }
    return res


def vrp(data=None):
    """
    Entry point of the program.

    Parameters
    ----------
    data: optional data model
    """
    # Instantiate the data problem.
    if not data:
        data = data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    logging.debug("m1")

    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        max_travel_distance,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add global span coefficient to avoid having just one truck doing everything
    distance_dimension.SetGlobalSpanCostCoefficient(globalSpanCostCoefficient)

    for node in range(1, len(data["distance_matrix"])):
        # loop over distances between each location
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty_value)

    logging.debug("m2")

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node"""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    logging.debug("m3")

    # Add Time Windows constraint.
    # Duration Callback
    def duration_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["duration_matrix"][from_node][to_node]

    time_evaluator_index = routing.RegisterTransitCallback(duration_callback)

    time = "Time"
    routing.AddDimension(
        time_evaluator_index,
        0,  # allow waiting time
        impossible * 10,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)

    logging.debug("m3a")

    # Add time window constraints for each location except depot.
    # print('time window - location')
    for location_idx, time_window in enumerate(data["time_windows_orders"]):
        location_idx += 1  # skip zero-distance point
        # print(location_idx, time_window)
        index = manager.NodeToIndex(location_idx)
        # print(index)

        if index > len(data["time_windows_orders"]):
            break

        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    logging.debug("m3b")

    # print('time window - trucks')
    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        # print(vehicle_id, index, data['time_windows_trucks'][vehicle_id])
        time_dimension.CumulVar(index).SetRange(
            data["time_windows_trucks"][vehicle_id][0],
            data["time_windows_trucks"][vehicle_id][1],
        )

    logging.debug("m3c")

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC
    )

    logging.debug("m5")

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    logging.debug("m6")

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

    else:
        logging.error("NO SOLUTION FOUND")

    return get_vehicle_data(data, manager, routing, solution)


sol = vrp(data)
sol

DEBUG:root:m1
DEBUG:root:m2
DEBUG:root:m3
DEBUG:root:m3a
DEBUG:root:m3b
DEBUG:root:m3c
DEBUG:root:m5
DEBUG:root:m6


Objective: 35035
Route for vehicle 0:
 1 Load(0) Time(300, 300) ->  4 Load(250) Time(338, 338) ->  5 Load(300) Time(409, 409) ->  3 Load(400) Time(472, 472) ->  2 Load(800) Time(534, 534) ->  0 Load(800), Time(534, 534)
Distance of the route: 35km
Load of the route: 800
Time of the route: 534min

Total distance of all routes: 35km
Total load of all routes: 800
Total time of all routes: 534min


{'objective': 35035,
 'vehicle_data': {0: {'route': [1, 4, 5, 3, 2, 0],
   'cum_capacity': [0, 250, 300, 400, 800, 800],
   'cum_distance': [0, 5, 20, 29, 35, 35],
   'cum_time': [300, 338, 409, 472, 534, 534]}}}

In [15]:
def get_ors_route_multiple_points(lnglatlist, ors_key):
    """
    Returns response from ORS on route information between points in `lnglatlist`.

    Parameters
    ----------
    lnglatlist: list of coordinates, which are also in list form
    ors_key: API key needed to link to ORS
    """

    body = {"coordinates": lnglatlist}

    headers = {
        "Accept": "application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8",
        "Authorization": ors_key,
        "Content-Type": "application/json; charset = utf-8",
    }
    response = requests.post(
        "https://api.openrouteservice.org/v2/directions/driving-hgv/geojson",
        json=body,
        headers=headers,
    )

    return response

In [16]:
# add offset for zero-distance point
points = [None].__add__(arr_points)

# add Long Lats
for key, val in sol["vehicle_data"].items():
    temp_route_lnglat = []
    for route_num in sol["vehicle_data"][key]["route"]:
        if route_num != 0:
            temp_route_lnglat.append(points[route_num])
    sol["vehicle_data"][key]["route_lnglat"] = temp_route_lnglat

# pull routes using ors and add to res
for key, val in sol["vehicle_data"].items():
    if len(sol["vehicle_data"][key]["route_lnglat"]) > 1:
        sol["vehicle_data"][key]["total_route_geojson"] = get_ors_route_multiple_points(
            sol["vehicle_data"][key]["route_lnglat"], ors_key
        ).json()["features"][0]["geometry"]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openrouteservice.org:443
DEBUG:urllib3.connectionpool:https://api.openrouteservice.org:443 "POST /v2/directions/driving-hgv/geojson HTTP/1.1" 200 7369


### Output: Optimized route order 

In [17]:
sol

{'objective': 35035,
 'vehicle_data': {0: {'route': [1, 4, 5, 3, 2, 0],
   'cum_capacity': [0, 250, 300, 400, 800, 800],
   'cum_distance': [0, 5, 20, 29, 35, 35],
   'cum_time': [300, 338, 409, 472, 534, 534],
   'route_lnglat': [[120.96398162841797, 14.634960174560547],
    [120.94569396972656, 14.657938957214355],
    [121.01588439941406, 14.583112716674805],
    [121.07904052734375, 14.612759590148926],
    [121.09098815917969, 14.576546669006348]],
   'total_route_geojson': {'coordinates': [[120.963967, 14.634951],
     [120.963995, 14.634908],
     [120.962721, 14.634052],
     [120.96264, 14.634237],
     [120.962621, 14.634284],
     [120.962412, 14.634723],
     [120.96221, 14.63516],
     [120.962199, 14.635185],
     [120.962016, 14.635559],
     [120.961814, 14.635985],
     [120.96162, 14.636196],
     [120.961551, 14.636255],
     [120.961172, 14.636551],
     [120.960697, 14.636934],
     [120.960636, 14.636985],
     [120.960354, 14.637204],
     [120.960178, 14.637338]

In [18]:
def opt_route(sol):
    """
    Returns list of optimized route, with elements of list corresponding to keys in input_dict

    Parameters
    ----------
    sol: __
    """
    base_opt = sol["vehicle_data"][0]["route"].copy()
    base_opt.remove(0)  # remove last item (0) as that's a dummy location
    opt_route = ["origin"] + [
        chr(char + 95) for char in base_opt[1:]
    ]  # convert numbers into letters from input_dict

    return opt_route


opt_route = opt_route(sol)
opt_route

['origin', 'c', 'd', 'b', 'a']

### Output: Route line

In [20]:
def route_line(route_order, input_dict):
    """
    Returns dictionary of lines in order, with geometry and other information

    Parameters:
    ----------
    route_order: List of route order, with elements in list corresponding to keys in 'input_dict'
    """

    route_dict = {}
    route_addresses = []

    # Append addresses according to route_order
    for i in route_order:
        if i == "origin":
            route_addresses.append(input_dict["starting_location"]["start_address"])
        else:
            route_addresses.append(input_dict["delivery_details"][i]["location"])

    # Turn list of addresses into list of coordinates
    geocoded_route = geocode_list(route_addresses)

    for i, pt in enumerate(geocoded_route):
        if i == len(geocoded_route) - 1:  # If last, don't do it
            break
        temp_route = [geocoded_route[i], geocoded_route[i + 1]]
        response = get_ors_route_multiple_points(temp_route, ors_key).json()
        distance = (
            response["features"][0]["properties"]["summary"]["distance"] / 1000
        )  # kilometers
        duration = (
            response["features"][0]["properties"]["summary"]["duration"] / 60
        )  # minutes
        line = response["features"][0]["geometry"]["coordinates"]

        route_dict[f"line_{i + 1}"] = {
            "starting_loc": route_order[i],
            "ending_loc": route_order[i + 1],
            "distance_km": distance,
            "duration_min": duration,
            "geometry": line,
        }

    return route_dict

#### Base route

In [21]:
route_order = ["origin"] + [k for k, v in del_dev.items() if v]
line_route = route_line(route_order, input_dict)
line_route

INFO:WazeRouteCalculator.WazeRouteCalculator:From: 69 J.Lianes Escoda, Manila, Metro Manila, Philippines - to: Manila, Philippines
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.waze.com:443
DEBUG:urllib3.connectionpool:https://www.waze.com:443 "GET /row-SearchServer/mozi?q=69+J.Lianes+Escoda%2C+Manila%2C+Metro+Manila%2C+Philippines&lang=eng&origin=livemap&lat=-35.281&lon=149.128 HTTP/1.1" 200 3162
DEBUG:WazeRouteCalculator.WazeRouteCalculator:Start coords: (14.634960174560547, 120.96398162841797)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.waze.com:443
DEBUG:urllib3.connectionpool:https://www.waze.com:443 "GET /row-SearchServer/mozi?q=Manila%2C+Philippines&lang=eng&origin=livemap&lat=-35.281&lon=149.128 HTTP/1.1" 200 2422
DEBUG:WazeRouteCalculator.WazeRouteCalculator:End coords: (14.602478981018066, 120.98548126220703)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.waze.com:443
DEBUG:urllib3.connectionpool:https://www.w

{'line_1': {'starting_loc': 'origin',
  'ending_loc': 'a',
  'distance_km': 22.5363,
  'duration_min': 37.13333333333333,
  'geometry': [[120.963967, 14.634951],
   [120.963995, 14.634908],
   [120.962721, 14.634052],
   [120.96264, 14.634237],
   [120.962621, 14.634284],
   [120.962412, 14.634723],
   [120.96221, 14.63516],
   [120.962199, 14.635185],
   [120.962016, 14.635559],
   [120.961814, 14.635985],
   [120.96162, 14.636196],
   [120.961551, 14.636255],
   [120.961172, 14.636551],
   [120.960697, 14.636934],
   [120.960636, 14.636985],
   [120.960354, 14.637204],
   [120.960178, 14.637338],
   [120.959903, 14.637546],
   [120.959848, 14.637588],
   [120.959713, 14.637694],
   [120.959331, 14.637995],
   [120.959013, 14.638244],
   [120.958883, 14.638347],
   [120.958499, 14.638637],
   [120.958192, 14.638898],
   [120.957998, 14.639066],
   [120.957676, 14.639436],
   [120.957547, 14.639568],
   [120.95738, 14.639774],
   [120.957171, 14.64005],
   [120.957143, 14.640087],
   [

#### Optimized route

In [24]:
line_route = route_line(opt_route, input_dict)
line_route

INFO:WazeRouteCalculator.WazeRouteCalculator:From: 69 J.Lianes Escoda, Manila, Metro Manila, Philippines - to: Manila, Philippines
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.waze.com:443
DEBUG:urllib3.connectionpool:https://www.waze.com:443 "GET /row-SearchServer/mozi?q=69+J.Lianes+Escoda%2C+Manila%2C+Metro+Manila%2C+Philippines&lang=eng&origin=livemap&lat=-35.281&lon=149.128 HTTP/1.1" 200 3162
DEBUG:WazeRouteCalculator.WazeRouteCalculator:Start coords: (14.634960174560547, 120.96398162841797)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.waze.com:443
DEBUG:urllib3.connectionpool:https://www.waze.com:443 "GET /row-SearchServer/mozi?q=Manila%2C+Philippines&lang=eng&origin=livemap&lat=-35.281&lon=149.128 HTTP/1.1" 200 2422
DEBUG:WazeRouteCalculator.WazeRouteCalculator:End coords: (14.602478981018066, 120.98548126220703)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.waze.com:443
DEBUG:urllib3.connectionpool:https://www.w

{'line_1': {'starting_loc': 'origin',
  'ending_loc': 'c',
  'distance_km': 5.0811,
  'duration_min': 8.84,
  'geometry': [[120.963967, 14.634951],
   [120.963995, 14.634908],
   [120.962721, 14.634052],
   [120.96264, 14.634237],
   [120.962621, 14.634284],
   [120.962412, 14.634723],
   [120.96221, 14.63516],
   [120.962199, 14.635185],
   [120.962016, 14.635559],
   [120.961814, 14.635985],
   [120.96162, 14.636196],
   [120.961551, 14.636255],
   [120.961172, 14.636551],
   [120.960697, 14.636934],
   [120.960636, 14.636985],
   [120.960354, 14.637204],
   [120.960178, 14.637338],
   [120.959903, 14.637546],
   [120.959848, 14.637588],
   [120.959713, 14.637694],
   [120.959331, 14.637995],
   [120.959013, 14.638244],
   [120.958883, 14.638347],
   [120.958499, 14.638637],
   [120.958192, 14.638898],
   [120.957998, 14.639066],
   [120.957676, 14.639436],
   [120.957547, 14.639568],
   [120.95738, 14.639774],
   [120.957171, 14.64005],
   [120.957143, 14.640087],
   [120.957096, 14